In [1]:
device_id = 0
device = 'cuda:' + str(device_id)
#device = 'cpu'

import time
import os
import torch

log_dir = os.path.join('./log/question_classifier', time.strftime('%Y%m%d_%H%M%S', time.localtime(time.time())))
print(log_dir)

./log/question_classifier/20210618_010247


In [2]:
from pkg.parse import *
from pkg.words import *
from problems import *

problems = [P1_1_1, P1_1_2, P1_1_3, P1_1_4, P1_1_5, P1_1_6, P1_1_7, P1_1_8, P1_1_9, P1_1_10, P1_1_11, P1_1_12, 
            P1_2_1, P1_2_2, P1_3_1, P1_4_1, 
            P2_1_1, P2_2_2, P2_3_1, 
            P3_1_1, P3_2_1, P3_2_2, P3_3_1, 
            P4_1_1, P4_2_1, P4_2_2, P4_3_1, 
            P5_1_1, P5_2_1, P5_3_1,
            P6_1_1, P6_3_1, P6_4_1,
            P7_1_1, P7_1_2, P7_3_1,
            P8_1_1, P8_2_1, P8_3_1, 
            P9_1_1, P9_2_1, P9_2_2, P9_3_1, P9_3_2]

In [3]:
from pkg.trainer_question_classifier import Hyper as hp

# dataset
hp.add_bos = False
hp.add_eos = False

hp.batch_size = None
hp.batch_type = 'normal'
hp.ds_batch_size = 256

# train
hp.num_workers = 2

hp.steps_log = 10
hp.steps_eval = 50
hp.steps_save = 10000

hp.weight_decay = 0.000001
hp.initial_lr = 0.0001
hp.final_lr = 0.00001
hp.lr_decay_factor = 0.99
hp.lr_patience = 300
hp.ema = 0.99
hp.grad_norm_max = 10.0

hp.adam_alpha = 2e-4
hp.adam_betas = (0.5, 0.9)
hp.adam_eps = 1e-6

hp.vocab_size = 512
hp.add_space_token = False

hp.num_problems = len(problems)
hp.problems = problems


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from pkg.dataset import ProblemDataset, QuestionDataset, read_questions, write_questions

dir_question = 'data/question'
_prefix = 'question_'
_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
path_questions = [os.path.join(dir_question, _prefix + str(i) + '.txt') for i in _ids]
metas = [read_questions(path) for path in path_questions]
meta = []
for m in metas:
    meta += m
len_train = int(len(meta) * 0.8)
meta_train = meta[:len_train]
meta_val = meta[len_train:]
print(len(meta), len(meta_train), len(meta_val))

2816000 2252800 563200


In [5]:
from pkg.vocab import Vocab, CharVocab, SPVocab
dir_token = 'tokenization'
filename = 'prob'
filename += '_' + str(hp.vocab_size)
if hp.add_space_token:
    filename += '_'
filename += '.model'
path_model = os.path.join(dir_token, filename)
print(path_model)
vocab = SPVocab(path_model)
print(vocab.vocab_size())

tokenization/prob_512.model
512


In [6]:
start = time.time()
dsTrain = QuestionDataset(meta_train, vocab, batch_size=hp.ds_batch_size)
dsVal = QuestionDataset(meta_val, vocab, batch_size=hp.ds_batch_size)
print(time.time() - start)

644.2300124168396


In [7]:
from pkg.models.model import QuestionClassifier
from pkg.models.config import Config
from pkg.models.extractor import AverageExtractor, RNNExtractor
from pkg.models.encoders_conv import ConvEncoder, HighwayEncoder
from pkg.models.embedding import Embed, Regressor

cfg = QuestionClassifier.default_config()
cfg.text_embed.num_symbols = hp.vocab_size
cfg.encoders[0].dropout_rate=0.5
cfg.regressor.num_symbols = hp.num_problems
cfg.extractor = RNNExtractor.default_config()
model = cfg.create_object().to(device)

In [8]:
from pkg.trainer_question_classifier import Trainer_QuestionClassifier

trainer = Trainer_QuestionClassifier(model, dsTrain, dsVal, hp=hp, log_dir=log_dir)

---------------------- hp -------------------

__module__ pkg.trainer
steps_log 10
steps_eval 50
steps_save 10000
save_model_only True
max_lr 0.001
base_lr 0.001
lr_decay_factor 0.99
lr_patience 300
scheduler_mode triangular2
step_size_up 10
ema 0.99
max_grad_norm 10.0
adam_alpha 0.0002
adam_betas (0.5, 0.9)
adam_eps 1e-06
weight_decay 1e-06
__dict__ <attribute '__dict__' of 'Hyper' objects>
__weakref__ <attribute '__weakref__' of 'Hyper' objects>
__doc__ None
add_bos False
add_eos False
batch_size None
batch_type normal
ds_batch_size 256
num_workers 2
initial_lr 0.0001
final_lr 1e-05
grad_norm_max 10.0
vocab_size 512
add_space_token False
num_problems 44
problems [<class 'problems.P1_1_1'>, <class 'problems.P1_1_2'>, <class 'problems.P1_1_3'>, <class 'problems.P1_1_4'>, <class 'problems.P1_1_5'>, <class 'problems.P1_1_6'>, <class 'problems.P1_1_7'>, <class 'problems.P1_1_8'>, <class 'problems.P1_1_9'>, <class 'problems.P1_1_10'>, <class 'problems.P1_1_11'>, <class 'problems.P1_1_12'>,

In [ ]:
trainer.train()

step: 10 train: 0.12909269332885742 elapsed, loss: 3.7471013
step: 20 train: 0.12599515914916992 elapsed, loss: 3.5824456
step: 30 train: 0.1262345314025879 elapsed, loss: 3.28863
step: 40 train: 0.12466096878051758 elapsed, loss: 2.9132555
step: 50 train: 0.12120389938354492 elapsed, loss: 2.3621979
step: 60 train: 0.11808276176452637 elapsed, loss: 1.9671142
step: 70 train: 0.12321853637695312 elapsed, loss: 1.7017059
step: 80 train: 0.12485313415527344 elapsed, loss: 1.4443362
step: 90 train: 0.12475323677062988 elapsed, loss: 1.3768853
step: 100 train: 0.12387537956237793 elapsed, loss: 1.2476236
step: 110 train: 0.11437439918518066 elapsed, loss: 1.1036048
step: 120 train: 0.131760835647583 elapsed, loss: 0.83038163
step: 130 train: 0.12621355056762695 elapsed, loss: 0.7873377
step: 140 train: 0.1207432746887207 elapsed, loss: 0.69633055
step: 150 train: 0.11683416366577148 elapsed, loss: 0.4981973
step: 160 train: 0.13048505783081055 elapsed, loss: 0.4476271
step: 170 train: 0.12

step: 1290 train: 0.12659263610839844 elapsed, loss: 9.451736e-06
step: 1300 train: 0.12119126319885254 elapsed, loss: 1.2407685e-05
step: 1310 train: 0.12518668174743652 elapsed, loss: 8.565613e-06
step: 1320 train: 0.1290111541748047 elapsed, loss: 8.79762e-06
step: 1330 train: 0.1235804557800293 elapsed, loss: 1.0195535e-05
step: 1340 train: 0.12596654891967773 elapsed, loss: 9.072186e-06
step: 1350 train: 0.1252121925354004 elapsed, loss: 6.5056624e-06
step: 1360 train: 0.12937331199645996 elapsed, loss: 1.05137005e-05
step: 1370 train: 0.14623188972473145 elapsed, loss: 5.5185124e-06
step: 1380 train: 0.13597321510314941 elapsed, loss: 5.479842e-06
step: 1390 train: 0.13138175010681152 elapsed, loss: 5.1855695e-06
step: 1400 train: 0.14192962646484375 elapsed, loss: 9.2358705e-06
step: 1410 train: 0.1336688995361328 elapsed, loss: 5.7019197e-06
step: 1420 train: 0.13150572776794434 elapsed, loss: 4.671831e-05
step: 1430 train: 0.1507399082183838 elapsed, loss: 7.2556313e-06
step: 

step: 2530 train: 0.13309884071350098 elapsed, loss: 3.723344e-06
step: 2540 train: 0.12346601486206055 elapsed, loss: 3.8179155e-06
step: 2550 train: 0.12983322143554688 elapsed, loss: 4.94462e-06
step: 2560 train: 0.12224102020263672 elapsed, loss: 3.7709033e-06
step: 2570 train: 0.11636543273925781 elapsed, loss: 4.108033e-06
step: 2580 train: 0.1274402141571045 elapsed, loss: 4.335217e-06
step: 2590 train: 0.11522626876831055 elapsed, loss: 3.936618e-06
step: 2600 train: 0.12332844734191895 elapsed, loss: 4.4562594e-06
step: 2610 train: 0.12139129638671875 elapsed, loss: 4.8386055e-06
step: 2620 train: 0.1410045623779297 elapsed, loss: 2.8265526e-06
step: 2630 train: 0.13256525993347168 elapsed, loss: 3.429573e-06
step: 2640 train: 0.1389310359954834 elapsed, loss: 3.6991007e-06
step: 2650 train: 0.13597989082336426 elapsed, loss: 3.0887163e-06
step: 2660 train: 0.12159919738769531 elapsed, loss: 3.616776e-06
step: 2670 train: 0.12690091133117676 elapsed, loss: 4.637366e-06
step: 2

step: 3770 train: 0.12884235382080078 elapsed, loss: 6.7197043e-06
step: 3780 train: 0.1383051872253418 elapsed, loss: 4.083816e-06
step: 3790 train: 0.12803030014038086 elapsed, loss: 6.0088555e-06
step: 3800 train: 0.13739514350891113 elapsed, loss: 5.835433e-06
step: 3810 train: 0.1329340934753418 elapsed, loss: 3.4915129e-06
step: 3820 train: 0.13950324058532715 elapsed, loss: 3.4924465e-06
step: 3830 train: 0.12243509292602539 elapsed, loss: 6.3645884e-06
step: 3840 train: 0.13419842720031738 elapsed, loss: 4.9875634e-06
step: 3850 train: 0.12719488143920898 elapsed, loss: 4.2379124e-06
step: 3860 train: 0.13112521171569824 elapsed, loss: 2.9615958e-06
step: 3870 train: 0.12811541557312012 elapsed, loss: 3.5948535e-06
step: 3880 train: 0.11988997459411621 elapsed, loss: 5.5370083e-06
step: 3890 train: 0.13707828521728516 elapsed, loss: 2.6100245e-06
step: 3900 train: 0.11949777603149414 elapsed, loss: 4.919192e-06
step: 3910 train: 0.13315701484680176 elapsed, loss: 4.862294e-05
s

step: 5010 train: 0.12532329559326172 elapsed, loss: 4.2426027e-06
step: 5020 train: 0.1294100284576416 elapsed, loss: 3.7913946e-06
step: 5030 train: 0.13141441345214844 elapsed, loss: 3.0803385e-06
step: 5040 train: 0.12769579887390137 elapsed, loss: 3.245647e-06
step: 5050 train: 0.1407604217529297 elapsed, loss: 2.874517e-06
step: 5060 train: 0.1333770751953125 elapsed, loss: 9.9915305e-06
step: 5070 train: 0.1313464641571045 elapsed, loss: 3.1455083e-06
step: 5080 train: 0.11784172058105469 elapsed, loss: 8.084118e-06
step: 5090 train: 0.11809325218200684 elapsed, loss: 3.9497127e-06
step: 5100 train: 0.12310457229614258 elapsed, loss: 6.8985814e-06
step: 5110 train: 0.12281489372253418 elapsed, loss: 4.403172e-06
step: 5120 train: 0.12765026092529297 elapsed, loss: 3.382987e-06
step: 5130 train: 0.12347245216369629 elapsed, loss: 3.8127619e-06
step: 5140 train: 0.1273946762084961 elapsed, loss: 0.00021448395
step: 5150 train: 0.11408615112304688 elapsed, loss: 9.163311e-06
step: 

...

step: 72190 train: 0.12280082702636719 elapsed, loss: 1.5081062e-05
step: 72200 train: 0.12162089347839355 elapsed, loss: 1.0105169e-05
step: 72210 train: 0.14031100273132324 elapsed, loss: 5.7154925e-06
step: 72220 train: 0.12807536125183105 elapsed, loss: 7.515496e-06
step: 72230 train: 0.13218092918395996 elapsed, loss: 4.6709883e-06
step: 72240 train: 0.1326758861541748 elapsed, loss: 4.55601e-06
step: 72250 train: 0.13180303573608398 elapsed, loss: 2.8917482e-06
step: 72260 train: 0.1393904685974121 elapsed, loss: 4.3458404e-06
step: 72270 train: 0.11934876441955566 elapsed, loss: 3.3094452e-06
step: 72280 train: 0.1266036033630371 elapsed, loss: 2.4586852e-06
step: 72290 train: 0.1238105297088623 elapsed, loss: 2.7096767e-06
step: 72300 train: 0.11904764175415039 elapsed, loss: 2.6915168e-06
step: 72310 train: 0.125579833984375 elapsed, loss: 0.00010684995
step: 72320 train: 0.13081884384155273 elapsed, loss: 9.816025e-06
step: 72330 train: 0.13026094436645508 elapsed, loss: 0.00

step: 73410 train: 0.12450361251831055 elapsed, loss: 2.446577e-06
step: 73420 train: 0.12343692779541016 elapsed, loss: 2.110373e-06
step: 73430 train: 0.12583637237548828 elapsed, loss: 2.3380808e-06
step: 73440 train: 0.12577557563781738 elapsed, loss: 2.7585606e-06
step: 73450 train: 0.13692259788513184 elapsed, loss: 2.1853423e-06
step: 73460 train: 0.1235661506652832 elapsed, loss: 3.3671881e-06
step: 73470 train: 0.1315748691558838 elapsed, loss: 2.3408716e-06
step: 73480 train: 0.1371145248413086 elapsed, loss: 1.9245754e-06
step: 73490 train: 0.12824749946594238 elapsed, loss: 2.0237608e-06
step: 73500 train: 0.12796831130981445 elapsed, loss: 2.718059e-06
step: 73510 train: 0.12868595123291016 elapsed, loss: 2.4782441e-06
step: 73520 train: 0.14250469207763672 elapsed, loss: 2.3152921e-05
step: 73530 train: 0.11680936813354492 elapsed, loss: 0.023878241
step: 73540 train: 0.1268172264099121 elapsed, loss: 8.12261e-05
step: 73550 train: 0.12909436225891113 elapsed, loss: 3.125

step: 74630 train: 0.12032580375671387 elapsed, loss: 3.3932604e-06
step: 74640 train: 0.13216924667358398 elapsed, loss: 2.6198036e-06
step: 74650 train: 0.1287672519683838 elapsed, loss: 2.224458e-06
step: 74660 train: 0.1317734718322754 elapsed, loss: 1.7019884e-06
step: 74670 train: 0.13087797164916992 elapsed, loss: 2.1029223e-06
step: 74680 train: 0.14074993133544922 elapsed, loss: 1.5725357e-06
step: 74690 train: 0.1431593894958496 elapsed, loss: 1.7895329e-06
step: 74700 train: 0.12916135787963867 elapsed, loss: 1.9306276e-06
step: 74710 train: 0.12381291389465332 elapsed, loss: 7.0685383e-06
step: 74720 train: 0.11940479278564453 elapsed, loss: 2.9322614e-06
step: 74730 train: 0.12837576866149902 elapsed, loss: 2.4628775e-06
step: 74740 train: 0.12355852127075195 elapsed, loss: 2.2337729e-06
step: 74750 train: 0.1285231113433838 elapsed, loss: 2.0670668e-06
step: 74760 train: 0.12194323539733887 elapsed, loss: 2.212353e-06
step: 74770 train: 0.12545323371887207 elapsed, loss: 

step: 75850 train: 0.13762617111206055 elapsed, loss: 1.9194508e-06
step: 75860 train: 0.1424882411956787 elapsed, loss: 1.4910455e-06
step: 75870 train: 0.14431047439575195 elapsed, loss: 1.5362145e-06
step: 75880 train: 0.13822102546691895 elapsed, loss: 1.6326065e-06
step: 75890 train: 0.127593994140625 elapsed, loss: 2.3338898e-06
step: 75900 train: 0.13719463348388672 elapsed, loss: 1.6437817e-06
step: 75910 train: 0.11535143852233887 elapsed, loss: 3.1241116e-06
step: 75920 train: 0.13086795806884766 elapsed, loss: 1.7723046e-06
step: 75930 train: 0.13447213172912598 elapsed, loss: 1.6307436e-06
step: 75940 train: 0.12359905242919922 elapsed, loss: 2.2905836e-06
step: 75950 train: 0.12115263938903809 elapsed, loss: 2.9592693e-06
step: 75960 train: 0.11685562133789062 elapsed, loss: 2.971376e-06
step: 75970 train: 0.12619948387145996 elapsed, loss: 2.4549608e-06
step: 75980 train: 0.13320064544677734 elapsed, loss: 2.155064e-06
step: 75990 train: 0.11336851119995117 elapsed, loss:

step: 77070 train: 0.1220712661743164 elapsed, loss: 4.149958e-06
step: 77080 train: 0.11947107315063477 elapsed, loss: 3.3946312e-06
step: 77090 train: 0.13407039642333984 elapsed, loss: 2.0354018e-06
step: 77100 train: 0.14262032508850098 elapsed, loss: 1.5511155e-06
step: 77110 train: 0.1317734718322754 elapsed, loss: 2.0004752e-06
step: 77120 train: 0.1407155990600586 elapsed, loss: 2.1057167e-06
step: 77130 train: 0.13442182540893555 elapsed, loss: 1.6763785e-06
step: 77140 train: 0.13609790802001953 elapsed, loss: 1.4090894e-06
step: 77150 train: 0.12752461433410645 elapsed, loss: 2.490351e-06
step: 77160 train: 0.13240337371826172 elapsed, loss: 1.5986134e-06
step: 77170 train: 0.13077783584594727 elapsed, loss: 1.9515815e-06
step: 77180 train: 0.13153862953186035 elapsed, loss: 2.12993e-06
step: 77190 train: 0.12267851829528809 elapsed, loss: 2.2430863e-06
